In [56]:
from scipy.io import arff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [62]:
data = arff.loadarff("KDDTrain+.arff")
data_test = arff.loadarff("KDDTest+.arff")

train_df = pd.DataFrame(data[0])
test_df = pd.DataFrame(data_test[0])

train_df.head()
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_df)
enc.categories_

[array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 4.2862e+04, 4.2888e+04,
        4.2908e+04]),
 array([b'icmp', b'tcp', b'udp'], dtype=object),
 array([b'IRC', b'X11', b'Z39_50', b'aol', b'auth', b'bgp', b'courier',
        b'csnet_ns', b'ctf', b'daytime', b'discard', b'domain',
        b'domain_u', b'echo', b'eco_i', b'ecr_i', b'efs', b'exec',
        b'finger', b'ftp', b'ftp_data', b'gopher', b'harvest',
        b'hostnames', b'http', b'http_2784', b'http_443', b'http_8001',
        b'imap4', b'iso_tsap', b'klogin', b'kshell', b'ldap', b'link',
        b'login', b'mtp', b'name', b'netbios_dgm', b'netbios_ns',
        b'netbios_ssn', b'netstat', b'nnsp', b'nntp', b'ntp_u', b'other',
        b'pm_dump', b'pop_2', b'pop_3', b'printer', b'private', b'red_i',
        b'remote_job', b'rje', b'shell', b'smtp', b'sql_net', b'ssh',
        b'sunrpc', b'supdup', b'systat', b'telnet', b'tftp_u', b'tim_i',
        b'time', b'urh_i', b'urp_i', b'uucp', b'uucp_path', b'vmnet',
        b'whois'], d

In [58]:
train_df['class'].value_counts(sort=True)


b'normal'     67343
b'anomaly'    58630
Name: class, dtype: int64

In [52]:
train_df['protocol_type'].groupby(train_df['class']).value_counts(sort=True)

class       protocol_type
b'anomaly'  b'tcp'           49089
            b'icmp'           6982
            b'udp'            2559
b'normal'   b'tcp'           53600
            b'udp'           12434
            b'icmp'           1309
Name: protocol_type, dtype: int64